In [ ]:
import pandas as pd
import json
import math
import random

# ==========================
# Geometry Functions
# ==========================
def generate_geometry_from_params(length, width, height, wwr):
    k = math.sqrt(wwr)
    L, W, H = float(length), float(width), float(height)
    
    floor = [(L, W, 0.0), (L, 0.0, 0.0), (0.0, 0.0, 0.0), (0.0, W, 0.0)]
    roof = [(0.0, W, H), (0.0, 0.0, H), (L, 0.0, H), (L, W, H)]
    wall_N = [(L, W, H), (L, W, 0.0), (0.0, W, 0.0), (0.0, W, H)]
    wall_E = [(L, 0.0, H), (L, 0.0, 0.0), (L, W, 0.0), (L, W, H)]
    wall_S = [(0.0, 0.0, H), (0.0, 0.0, 0.0), (L, 0.0, 0.0), (L, 0.0, H)]
    wall_W = [(0.0, W, H), (0.0, W, 0.0), (0.0, 0.0, 0.0), (0.0, 0.0, H)]

    def centered_window_on_wall(axis_fixed, fixed_val, run_len, height_wall):
        win_w, win_h = k * run_len, k * height_wall
        run_min, run_max = (run_len - win_w) / 2.0, (run_len - win_w) / 2.0 + win_w
        z_min, z_max = (height_wall - win_h) / 2.0, (height_wall - win_h) / 2.0 + win_h
        if axis_fixed == 'y':
            return [(run_max, fixed_val, z_max), (run_max, fixed_val, z_min),
                    (run_min, fixed_val, z_min), (run_min, fixed_val, z_max)]
        else:
            return [(fixed_val, run_min, z_max), (fixed_val, run_min, z_min),
                    (fixed_val, run_max, z_min), (fixed_val, run_max, z_max)]

    win_N = centered_window_on_wall('y', W, L, H)
    win_E = centered_window_on_wall('x', L, W, H)
    win_S = centered_window_on_wall('y', 0.0, L, H)
    win_W = centered_window_on_wall('x', 0.0, W, H)

    return {
        "floor": floor,
        "roof": roof,
        "walls": {
            "Wall 0.00:a": wall_N,
            "Wall 90.00:a": wall_E,
            "Wall 180.00:a": wall_S,
            "Wall 270.00:a": wall_W
        },
        "windows": {
            "Wall 0.00:a - Sub:a": win_N,
            "Wall 90.00:a - Sub:a": win_E,
            "Wall 180.00:a - Sub:a": win_S,
            "Wall 270.00:a - Sub:a": win_W
        }
    }

def fmt_triplet(p):
    return f"{p[0]:.2f},{p[1]:.2f},{p[2]:.2f}"

def join_quad(quad):
    return ", ".join([fmt_triplet(p) for p in quad])

def idf_from_geometry(length, width, height, wwr, names=None):
    if names is None:
        names = {"zone": "zone1.00", "space": "zone1.00_Space", "spacelist": "program_so"}
    Z = float(height)
    V = float(length) * float(width) * float(height)
    geom = generate_geometry_from_params(length, width, height, wwr)

    lines = []
    lines.append(f"Zone, {names['zone']}, , 0.00, 0.00, 0.00, , 1.00, {Z:.2f}, {V:.2f}, , , , Yes;")
    lines.append(f"Space, {names['space']}, {names['zone']}, , , , {names['spacelist']};")
    lines.append(f"SpaceList, {names['spacelist']}, {names['space']};")

    floor = geom["floor"]
    lines.append(f"BuildingSurface:Detailed, {names['space']} - Floor:a, Floor, Typical Insulated Carpeted 6.00in Slab Floor, "
                 f"{names['zone']}, {names['space']}, Ground, , NoSun, NoWind, , , {join_quad(floor)};")
    roof = geom["roof"]
    lines.append(f"BuildingSurface:Detailed, {names['space']} - RoofCeiling:a, Roof, Typical IEAD Roof-R16.00, "
                 f"{names['zone']}, {names['space']}, Outdoors, , SunExposed, WindExposed, , , {join_quad(roof)};")
    for wall_name, quad in geom["walls"].items():
        lines.append(f"BuildingSurface:Detailed, {names['space']} - {wall_name}, Wall, Typical Insulated Exterior Mass Wall-R7.00, "
                     f"{names['zone']}, {names['space']}, Outdoors, , SunExposed, WindExposed, , , {join_quad(quad)};")
    for sub_name, quad in geom["windows"].items():
        parent = sub_name.split(" - Sub:")[0]
        lines.append(f"FenestrationSurface:Detailed, {names['space']} - {sub_name}, Window, "
                     f"U 0.57 SHGC 0.39 Dbl Ref-D Clr 6.00mm/6.00mm Air, "
                     f"{names['space']} - {parent}, , , , , , {join_quad(quad)};")
    return "\n".join(lines)

# ==========================
# CSV → JSONL Dataset Generator
# ==========================
TEMPLATES = [
    # "Generate the geometric objects of an EnergyPlus IDF with Length {L:.2f} m, Width {W:.2f} m, Height {H:.2f} m, and WWR {WWR:.2f}.",
    "Create the geometric objects of an IDF for a building: Length {L:.2f} m, Width {W:.2f} m, Height {H:.2f} m, Window-to-Wall Ratio {WWR:.2f}.",
    # "Please make the geometric objects of an EnergyPlus model. Dimensions: {L:.2f} by {W:.2f} meters, height {H:.2f} m, WWR {WWR:.2f}.",
]

def build_instruction(row):
    template = random.choice(TEMPLATES)
    return template.format(**row)

def build_reasoning(row):
    L, W, H, WWR = row["L"], row["W"], row["H"], row["WWR"]
    k = math.sqrt(WWR)

    # Window sizes
    win_w_N = k * L
    win_h_N = k * H
    win_w_E = k * W
    win_h_E = k * H

    # return (
    #     f"The building has length L = {L:.2f} m, width W = {W:.2f} m, height H = {H:.2f} m, and a window-to-wall ratio WWR = {WWR:.2f}. "
    #     f"The window scale factor is sqrt(WWR) = {k:.4f}, which is used to compute window width and height while preserving wall aspect ratio. "
    #     f"\n\nThe floor lies at z=0 with coordinates (L,W,z)=({L:.2f},{W:.2f},0.00), (L,0,z)=({L:.2f},0.00,0.00), (0,0,z)=(0,0,0) and (0,W,z)=(0.00,{W:.2f},0.00). "
    #     f"The roof lies at z={H:.2f} with coordinates (0,W,z)=(0.00,{W:.2f},{H:.2f}), (0,0,z)=(0.00,0.00,{H:.2f}), (L,0,z)=({L:.2f},0.00,{H:.2f}), and (L,W,z)=({L:.2f},{W:.2f},{H:.2f}). "
    #     f"\n\nThe north wall is at y={W:.2f} with corners (L,W,H)=({L:.2f},{W:.2f},{H:.2f}), (L,W,0), (0,W,0), (0,W,H). "
    #     f"The east wall is at x={L:.2f} with corners (L,0,H), (L,0,0), (L,W,0), (L,W,H). "
    #     f"The south wall is at y=0 with corners (0,0,H), (0,0,0), (L,0,0), (L,0,H). "
    #     f"The west wall is at x=0 with corners (0,W,H), (0,W,0), (0,0,0), (0,0,H). "
    #     f"\n\nEach wall has a centered window: "
    #     f"\n- North wall have length of {L:.2f} and height of {H:.2f}, window width = k*length = {win_w_N:.2f}, window height = k*height = {win_h_N:.2f}. "
    #     f"The window is centered, with x ranging from (L-k*length)/2={(L-win_w_N)/2:.2f} to (L+k*length)/2={(L+win_w_N)/2:.2f}, and z ranging from (H-k*height)/2={(H-win_h_N)/2:.2f} to (H+k*height)/2={(H+win_h_N)/2:.2f}. "
    #     f"\n- East wall have width of {W:.2f} and height of {H:.2f}, window width = k*width = {win_w_E:.2f}, window height = k*height = {win_h_E:.2f}. "
    #     f"The window is centered, with y ranging from (W-k*width)/2={(W-win_w_E)/2:.2f} to (W+k*width)/2={(W+win_w_E)/2:.2f}, and z ranging from (H-k*height)/2={(H-win_h_E)/2:.2f} to (H+k*height)/2={(H+win_h_E)/2:.2f}. "
    #     f"\n- South wall is identical in dimensions to north, window coordinates mirror accordingly at y=0. "
    #     f"\n- West wall is identical in dimensions to east, window coordinates mirror accordingly at x=0. "
    #     f"\n\nThe geometry dictionary returned is: "
    #     f"{{'floor': floor, 'roof': roof, 'walls': {{'Wall 000:a': wall_N, 'Wall 090:a': wall_E, 'Wall 180:a': wall_S, 'Wall 270:a': wall_W}}, "
    #     f"'windows': {{'Wall 000:a - Sub:a': win_N, 'Wall 090:a - Sub:a': win_E, 'Wall 180:a - Sub:a': win_S, 'Wall 270:a - Sub:a': win_W}}}}."
    # )

    # return (
    #     f"The building has length L = {L:.2f} m, width W = {W:.2f} m, height H = {H:.2f} m, and a window-to-wall ratio WWR = {WWR:.2f}. "
    #     f"The window scale factor is sqrt(WWR) = {k:.4f}, calculated as sqrt({WWR:.2f}) = {k:.4f}. This factor is used to compute window width and height while preserving wall aspect ratio. "
    #     f"\n\nThe floor lies at z=0 with coordinates (L,W,z)=({L:.2f},{W:.2f},0.00), (L,0,z)=({L:.2f},0.00,0.00), (0,0,z)=(0,0,0), and (0,W,z)=(0.00,{W:.2f},0.00). "
    #     f"The roof lies at z={H:.2f} with coordinates (0,W,z)=(0.00,{W:.2f},{H:.2f}), (0,0,z)=(0.00,0.00,{H:.2f}), (L,0,z)=({L:.2f},0.00,{H:.2f}), and (L,W,z)=({L:.2f},{W:.2f},{H:.2f}). "
    #     f"\n\nThe north wall is at y={W:.2f} with corners (L,W,H)=({L:.2f},{W:.2f},{H:.2f}), (L,W,0), (0,W,0), (0,W,H). "
    #     f"The east wall is at x={L:.2f} with corners (L,0,H), (L,0,0), (L,W,0), (L,W,H). "
    #     f"The south wall is at y=0 with corners (0,0,H), (0,0,0), (L,0,0), (L,0,H). "
    #     f"The west wall is at x=0 with corners (0,W,H), (0,W,0), (0,0,0), (0,0,H). "
    #     f"\n\nEach wall has a centered window with calculations as follows: "
    #     f"\n- North wall: length={L:.2f}, height={H:.2f}, window width = k*length = {k:.4f}*{L:.2f} = {win_w_N:.2f}, "
    #     f"window height = k*height = {k:.4f}*{H:.2f} = {win_h_N:.2f}. "
    #     f"Window x-coordinate range: (L-win_w_N)/2 = ({L:.2f}-{win_w_N:.2f})/2 = {(L-win_w_N)/2:.2f} to "
    #     f"(L+win_w_N)/2 = ({L:.2f}+{win_w_N:.2f})/2 = {(L+win_w_N)/2:.2f}. "
    #     f"Window z-coordinate range: (H-win_h_N)/2 = ({H:.2f}-{win_h_N:.2f})/2 = {(H-win_h_N)/2:.2f} to "
    #     f"(H+win_h_N)/2 = ({H:.2f}+{win_h_N:.2f})/2 = {(H+win_h_N)/2:.2f}. "
    #     f"\n- East wall: width={W:.2f}, height={H:.2f}, window width = k*width = {k:.4f}*{W:.2f} = {win_w_E:.2f}, "
    #     f"window height = k*height = {k:.4f}*{H:.2f} = {win_h_E:.2f}. "
    #     f"Window y-coordinate range: (W-win_w_E)/2 = ({W:.2f}-{win_w_E:.2f})/2 = {(W-win_w_E)/2:.2f} to "
    #     f"(W+win_w_E)/2 = ({W:.2f}+{win_w_E:.2f})/2 = {(W+win_w_E)/2:.2f}. "
    #     f"Window z-coordinate range: (H-win_h_E)/2 = ({H:.2f}-{win_h_E:.2f})/2 = {(H-win_h_E)/2:.2f} to "
    #     f"(H+win_h_E)/2 = ({H:.2f}+{win_h_E:.2f})/2 = {(H+win_h_E)/2:.2f}. "
    #     f"\n- South wall is identical to north; coordinates mirror at y=0. "
    #     f"\n- West wall is identical to east; coordinates mirror at x=0. "
    #     f"\n\nThe geometry dictionary returned is: "
    #     f"{{'floor': floor, 'roof': roof, 'walls': {{'Wall 000:a': wall_N, 'Wall 090:a': wall_E, 'Wall 180:a': wall_S, 'Wall 270:a': wall_W}}, "
    #     f"'windows': {{'Wall 000:a - Sub:a': win_N, 'Wall 090:a - Sub:a': win_E, 'Wall 180:a - Sub:a': win_S, 'Wall 270:a - Sub:a': win_W}}}}."
    # )

    # return(
    #     f"Step 1: Building dimensions are L={L:.2f} m, W={W:.2f} m, H={H:.2f} m, WWR={WWR:.2f}.",
    #     f"Step 2: Floor corners at z=0: (L,W,0), (L,0,0), (0,0,0), (0,W,0).",
    #     f"Step 3: Roof corners at z=H: (0,W,H), (0,0,H), (L,0,H), (L,W,H).",
    #     f"Step 4: Compute window scale factor: k = sqrt(WWR) = sqrt({WWR:.2f}) = {k:.4f}.",
    #     f"Step 5: North wall window width = k * L = {k:.4f} * {L:.2f} = {win_w_N:.2f} m.",
    #     f"Step 6: North wall window height = k * H = {k:.4f} * {H:.2f} = {win_h_N:.2f} m.",
    #     f"Step 7: North wall window x-range: (L - win_w_N)/2 = ({L:.2f} - {win_w_N:.2f})/2 = {(L - win_w_N)/2:.2f} m "
    #     f"to (L + win_w_N)/2 = ({L:.2f} + {win_w_N:.2f})/2 = {(L + win_w_N)/2:.2f} m.",
    #     f"Step 8: North wall window z-range: (H - win_h_N)/2 = ({H:.2f} - {win_h_N:.2f})/2 = {(H - win_h_N)/2:.2f} m "
    #     f"to (H + win_h_N)/2 = ({H:.2f} + {win_h_N:.2f})/2 = {(H + win_h_N)/2:.2f} m.",
    #     f"Step 9: East wall window width = k * W = {k:.4f} * {W:.2f} = {win_w_E:.2f} m.",
    #     f"Step 10: East wall window height = k * H = {k:.4f} * {H:.2f} = {win_h_E:.2f} m.",
    #     f"Step 11: East wall window y-range: (W - win_w_E)/2 = ({W:.2f} - {win_w_E:.2f})/2 = {(W - win_w_E)/2:.2f} m "
    #     f"to (W + win_w_E)/2 = ({W:.2f} + {win_w_E:.2f})/2 = {(W + win_w_E)/2:.2f} m.",
    #     f"Step 12: East wall window z-range: (H - win_h_E)/2 = ({H:.2f} - {win_h_E:.2f})/2 = {(H - win_h_E)/2:.2f} m "
    #     f"to (H + win_h_E)/2 = ({H:.2f} + {win_h_E:.2f})/2 = {(H + win_h_E)/2:.2f} m.",
    #     f"Step 13: South wall mirrors North wall at y=0; coordinates are identical but flipped.",
    #     f"Step 14: West wall mirrors East wall at x=0; coordinates are identical but flipped.",
    #     f"Step 15: Geometry dictionary returned: "
    #     f"{{'floor': floor, 'roof': roof, 'walls': {{'Wall 000:a': wall_N, 'Wall 090:a': wall_E, "
    #     f"'Wall 180:a': wall_S, 'Wall 270:a': wall_W}}, 'windows': {{'Wall 000:a - Sub:a': win_N, "
    #     f"'Wall 090:a - Sub:a': win_E, 'Wall 180:a - Sub:a': win_S, 'Wall 270:a - Sub:a': win_W}}}}."
    # )

    return(
        f"Step 1: The building has dimensions L={L:.2f} m, W={W:.2f} m, H={H:.2f} m, with window-to-wall ratio WWR={WWR:.2f}.",
        f"Step 2: Calculate window scale factor: k = sqrt(WWR) = sqrt({WWR:.2f}) = {k:.4f}.",

        # Floor first
        f"Step 3: Define floor corners at z=0: (L,W,0)=({L:.2f},{W:.2f},0.00), (L,0,0)=({L:.2f},0.00,0.00), "
        f"(0,0,0)=(0.00,0.00,0.00), (0,W,0)=(0.00,{W:.2f},0.00).",

        # Roof next
        f"Step 4: Define roof corners at z=H: (0,W,H)=(0.00,{W:.2f},{H:.2f}), (0,0,H)=(0.00,0.00,{H:.2f}), "
        f"(L,0,H)=({L:.2f},0.00,{H:.2f}), (L,W,H)=({L:.2f},{W:.2f},{H:.2f}).",

        # Walls
        f"Step 5: North wall at y={W:.2f} with corners: (L,W,H)=({L:.2f},{W:.2f},{H:.2f}), (L,W,0), (0,W,0), (0,W,H).",
        f"Step 6: East wall at x={L:.2f} with corners: (L,0,H), (L,0,0), (L,W,0), (L,W,H).",
        f"Step 7: South wall at y=0 with corners: (0,0,H), (0,0,0), (L,0,0), (L,0,H).",
        f"Step 8: West wall at x=0 with corners: (0,W,H), (0,W,0), (0,0,0), (0,0,H).",

        # Windows with math
        f"Step 9: North wall window: width = k * L = {k:.4f} * {L:.2f} = {win_w_N:.2f} m, height = k * H = {k:.4f} * {H:.2f} = {win_h_N:.2f} m.",
        f"Step 10: North wall window coordinates: x-range = (L - win_w_N)/2 = ({L:.2f} - {win_w_N:.2f})/2 = {(L-win_w_N)/2:.2f} "
        f"to (L + {win_w_N:.2f})/2 = {(L + win_w_N)/2:.2f}, z-range = (H - win_h_N)/2 = ({H:.2f} - {win_h_N:.2f})/2 = {(H-win_h_N)/2:.2f} "
        f"to (H + {win_h_N:.2f})/2 = {(H + win_h_N)/2:.2f}.",

        f"Step 11: East wall window: width = k * W = {k:.4f} * {W:.2f} = {win_w_E:.2f} m, height = k * H = {k:.4f} * {H:.2f} = {win_h_E:.2f} m.",
        f"Step 12: East wall window coordinates: y-range = (W - win_w_E)/2 = ({W:.2f} - {win_w_E:.2f})/2 = {(W-win_w_E)/2:.2f} "
        f"to (W + {win_w_E:.2f})/2 = {(W + win_w_E)/2:.2f}, z-range same as north wall: {(H-win_h_E)/2:.2f} to {(H+win_h_E)/2:.2f}.",

        f"Step 13: South wall mirrors north wall at y=0; West wall mirrors east wall at x=0.",

        f"Step 14: The geometry dictionary returned is: "
        f"{{'floor': floor, 'roof': roof, 'walls': {{'Wall 000:a': wall_N, 'Wall 090:a': wall_E, "
        f"'Wall 180:a': wall_S, 'Wall 270:a': wall_W}}, 'windows': {{'Wall 000:a - Sub:a': win_N, "
        f"'Wall 090:a - Sub:a': win_E, 'Wall 180:a - Sub:a': win_S, 'Wall 270:a - Sub:a': win_W}}}}."
    )
    

def make_example(row):
    instruction = build_instruction(row)
    reasoning = build_reasoning(row)
    idf = idf_from_geometry(row["L"], row["W"], row["H"], row["WWR"], names={
        "zone": "zone_01",
        "space": "zone_01_Space",
        "spacelist": "program_01"
    })
    return {"user": instruction, "reasoning": reasoning, "assistant": idf}

def csv_to_jsonl(input_csv, output_jsonl):
    df = pd.read_csv(input_csv)
    required_cols = {"ID", "L", "W", "H", "WWR"}
    if not required_cols.issubset(df.columns):
        raise ValueError(f"CSV must contain columns: {required_cols}")
    examples = [make_example(row) for _, row in df.iterrows()]
    with open(output_jsonl, "w", encoding="utf-8") as f:
        for ex in examples:
            f.write(json.dumps(ex, ensure_ascii=False) + "\n")
    print(f"Saved {len(examples)} examples to {output_jsonl}")


In [ ]:
# ==========================
# USAGE
# ==========================
csv_to_jsonl(r"C:\Users\Jayedi Aman\OneDrive - University of Missouri\Desktop\LLM\idf_rectangle/data.csv", 
             "reasoning_dataset3.jsonl")